# Exercice 3. [OPTIONNEL] Detection d'objet par regression de boîte englobante 

Dans cette partie, nous aborderons la détection par régression de la boîte englobante.

L'échantillon d'entrée n'est plus l'image recadrée, mais l'image entière.

L'étiquette ne correspond plus uniquement à la classe de l'objet, mais à sa classe plus sa position dans l'image. 
Nous allons normaliser cette position dans l'intervalle [0..1] par rapport à la taille de l'image.

Pour les premières expériences, nous ne considérerons que les instances de la classe __éléphant__. Par conséquent, nous ne traiterons que les images contenant des éléphants et nous ne conserverons que la première instance au sein de l'image.

Pour chaque instance d'elephant nous effectuerons plusieurs découpages autour de l'objet afin d'augmenter le nombre de données disponibles pour l'apprentissage, mais également pour travailler sur des données où l'objet occupe une place importante dans l'image. Cela permettra de simuler le travail au niveau d'une cellule dans une grande image.

La fonction ci-dessous permet de prendre en compte cette particularité.

**Q1** [Optionnel] Vous pourriez la modifier pour eventuellement ajouter du bruit gaussian autour de l'objet à detecter pour flouter davantage les éléments non pertinent.

In [ ]:
import os
import csv
import cv2
import numpy as np
import random
import math
import matplotlib.pyplot as plt


train_path="../tp/tp3/toy-dataset/train/"

def load_instances_of_one_class(imgs_path,label,max_samples=np.iinfo(np.int32),W=128,H=128,samples_per_instance=1):
    y_obj=[]
    x_obj=[]
    imgs_files = os.listdir( imgs_path+"images/" )
    for i,img_file in enumerate(imgs_files):
        if (len(x_obj)==max_samples):
            break
        
        label_file=img_file[:-4]+".txt"
        labels=csv.reader(open(imgs_path+"labels/"+label_file,"r"),delimiter=' ')    
        img_init=cv2.imread(imgs_path+"images/"+img_file)
        rows = list(labels)
        if len(rows)>0:
            for j,row in enumerate(rows):
                if (int(row[0]) != label):
                    continue
            
                bbox=np.array(row[1:],dtype=np.float32) if row[0]==1 else np.array(row[1:],dtype=np.float32)
                w=int(bbox[2]*img_init.shape[0]*2)
                h=int(bbox[3]*img_init.shape[1]*2)
                
                #should the annotation be out of screen, correct x0,y0,x1,y1
                x0=max(0,int(math.trunc(bbox[0]*img_init.shape[0]-w/2)))
                y0=max(0,int(math.trunc(bbox[1]*img_init.shape[1]-h/2)))
                x1=min(img_init.shape[0],int(math.trunc(x0+w)))
                y1=min(img_init.shape[1],int(math.trunc(y0+h)))
                h=y1-y0
                w=x1-x0
                #cv2.rectangle(img_init,(x0,y0),(x0+w,y0+h),4) # to check that we conserve well the initial annotations
                
                if (H-h<=5 or W-w<=5): # not to much space to randomly crop
                    #print("skipping due to the initial important size of the object in the image")
                    continue

                for s in range(samples_per_instance):
                    if (len(x_obj)==max_samples):
                        break
                        
                    startx=x0-W+w if x0-W+w>0 else 0
                    endx=x0
                    starty=y0-W+w if y0-W+w>0 else 0
                    endy=y0
                    
                    x_crop=random.randint(startx,endx)
                    y_crop=random.randint(starty,endy)
                    
                    x1=min(img_init.shape[0],int(math.trunc(x_crop+W)))
                    y1=min(img_init.shape[1],int(math.trunc(y_crop+H)))
                    
                    if (x1-x_crop != W or y1-y_crop != H):
                        continue
                    x_obj.append(np.copy(img_init[y_crop:y1,x_crop:x1]))
                    y_obj.append([row[0],x0-x_crop,y0-y_crop,x0-x_crop+w,y0-y_crop+h])
                    
    return x_obj,y_obj

def layout_images(images, n_rows=None, n_cols=None, border_size=2):
    height, width, n_channels = images[0].shape
    height, width = height + border_size, width + border_size
    if n_cols is None or n_rows is None:
        n_cols = int(np.sqrt(len(images)))
        if n_cols**2 < len(images):
            n_cols += 1
        n_rows = n_cols
    output_img = np.zeros((n_rows*height+border_size, n_cols*width+border_size, n_channels), dtype=images[0].dtype)
    for i, p in enumerate(images):
        r, c = i//n_cols, i%n_cols
        output_img[border_size+r*height:(r+1)*height, border_size+c*width:(c+1)*width] = p
    return output_img

Vous pouvez visualiser quelques instances avec le code ci-dessous

In [ ]:
label_elephant=4
label_start=label_elephant
label_end=label_elephant+1
nb_instances=36
nb_samples_par_instance=9
nb_cols=9

for i in range(label_start,label_end):
    x,y=load_instances_of_one_class(train_path,i,nb_instances,128,128,nb_samples_par_instance)
    for j in range(nb_instances):
        cv2.rectangle(x[j],(y[j][1],y[j][2]),(y[j][3],y[j][4]),5)
    plt.imshow(layout_images(x[:nb_instances],int(nb_instances/nb_cols)+1,nb_cols))
    plt.pause(1)

Nous devons maintenant construire le réseau pour la régression des boîtes englobantes. Nous pouvons partir de la structure du réseau utilisée dans l'exercice **1** et modifier la dernière couche dense afin de prédire : les quatre coordonnées de la boîte englobante.
Nous présenterons uniquement des images contenant un éléphant, ainsi il n'est pas nécessaire de garder l'information de la présence ou de l'absence de l'objet.

La loss sera donc constituée uniquement par la MeanSquaredError.

**Q2** Proposez dans un premier temps une architecture simple (quelques couches de convolutions, pooling)

**Q3** Evaluez la performance en termes de MSE sur l'ensemble de test

**Q4** Considerons qu'une détection est correcte si on observe un IoU suppérieur à 0.5.

Evaluez l'accuracy et le rappel sur l'ensemble de test.

Illustrez quelques examples de TP et FP